# Ensemble Learning

## Weighted Mean Ensemble

In [1]:
from models.ensemble_weighted_mean import EnsembleWeightedMean

ensemble_direct_optimization = EnsembleWeightedMean(optimization=True)
ensemble_direct_optimization_with_features = EnsembleWeightedMean(optimization=True, with_features=True)
ensemble_weigthed_mean = EnsembleWeightedMean(optimization=False)

data = 'ensemble-pd-hek293t-pe2.csv'

ensemble_direct_optimization.fit(data)
ensemble_direct_optimization.test(data)

/home/peiheng/anaconda3/envs/dl/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:54:03] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:197: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  predictions = torch.tensor([ridge_pred, xgboos

torch.Size([4503, 4])
torch.Size([4503, 4])


/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:211: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['mlp'].append(pearsonr(mlp_pred, target)[0])
/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:212: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['ensemble'].append(pearsonr(ensemble_predictions, target)[0])


Fold 1 Pearson: Ridge: 0.1142255840063799, XGBoost: -0.11417136543776873, Random Forest: 0.2829573785816031, MLP: [nan nan nan ... nan nan nan], Ensemble: [nan nan nan ... nan nan nan]
torch.Size([4605, 4])
torch.Size([4605, 4])


/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:211: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['mlp'].append(pearsonr(mlp_pred, target)[0])
/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:212: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['ensemble'].append(pearsonr(ensemble_predictions, target)[0])


...

/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:211: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['mlp'].append(pearsonr(mlp_pred, target)[0])
/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:212: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['ensemble'].append(pearsonr(ensemble_predictions, target)[0])


Fold 4 Pearson: Ridge: 0.08323622715467623, XGBoost: 0.30313542927279924, Random Forest: 0.2685319908887771, MLP: [nan nan nan ... nan nan nan], Ensemble: [nan nan nan ... nan nan nan]
torch.Size([4511, 4])
torch.Size([4511, 4])


/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:211: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['mlp'].append(pearsonr(mlp_pred, target)[0])
/home/peiheng/development/ox-dissertation/models/ensemble_weighted_mean.py:212: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  performances_pearson['ensemble'].append(pearsonr(ensemble_predictions, target)[0])


Fold 5 Pearson: Ridge: 0.1051749620109419, XGBoost: 0.2587617603004073, Random Forest: 0.257697786393736, MLP: [nan nan nan ... nan nan nan], Ensemble: [nan nan nan ... nan nan nan]


(defaultdict(list,
             {'ridge': [0.1142255840063799,
               0.1072743346283872,
               0.08827374421561401,
               0.08323622715467623,
               0.1051749620109419],
              'xgboost': [-0.11417136543776873,
               -0.13005880404980985,
               0.03974107720564746,
               0.30313542927279924,
               0.2587617603004073],
              'random_forest': [0.2829573785816031,
               0.28972199581239283,
               0.28608888009985295,
               0.2685319908887771,
               0.257697786393736],
              'mlp': [array([nan, nan, nan, ..., nan, nan, nan]),
               array([nan, nan, nan, ..., nan, nan, nan]),
               array([nan, nan, nan, ..., nan, nan, nan]),
               array([nan, nan, nan, ..., nan, nan, nan]),
               array([nan, nan, nan, ..., nan, nan, nan])],
              'ensemble': [array([nan, nan, nan, ..., nan, nan, nan]),
               array([nan, nan, n